In [1]:
import requests as rq
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import os
import datetime
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [187]:
%pip install dotenv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [35 lines of output]
           ---------------------------------------- 0.0/145.4 kB ? eta -:--:--
           ----- ------------------------------- 20.5/145.4 kB 640.0 kB/s eta 0:00:01
           --------------- --------------------- 61.4/145.4 kB 812.7 kB/s eta 0:00:01
           -------------------------------------- 145.4/145.4 kB 1.2 MB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Installing backend dependencies: started
        Installing backend dependencies: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: s

In [168]:
# named_tuple = time.localtime() # get struct_time
# time_string = time.strftime("%dD%mM%YY", named_tuple)
# print(time.strftime("%dD", named_tuple))
# print(time_string)
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
time_string = today.strftime("%dD%mM%YY")
ytime_string = yesterday.strftime("%dD%mM%YY")
print(time_string , ytime_string)


06D05M2024Y 05D05M2024Y


In [169]:
my_path = os.getcwd()
api_key = API_KEY

'''
Title and abstract
Document classification codes
Date of publication
Affiliations
Citations / references
Keywords
'''

# setup the HTTP headers
header = {"Accept": "application/json", "X-ELS-APIKey": api_key}
header2 = {"Accept": "*/xml", "X-ELS-APIKey": api_key}
header3 = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"}

# Break down the variables for the search query
api_url = "https://api.elsevier.com/content/search/scopus?query="
pubyear = ['2018','2019','2020','2021','2022','2023','2024']



#create folder a day
day_folder = my_path+'\\research_csv\\'+time_string
yesterday_folder = my_path+'\\research_csv\\'+ytime_string
if not os.path.exists(day_folder):
    os.mkdir(day_folder)    
#yesterday paper info and yesterday ref info
ypaper_info = pd.DataFrame()
yref_info = pd.DataFrame()

if os.path.exists(yesterday_folder):
    if os.path.exists(yesterday_folder+'\\paper_info.csv'):
        ypaper_info = pd.read_csv(yesterday_folder+'\\paper_info.csv')
    if os.path.exists(yesterday_folder+'\\ref_info.csv'):
        yref_info = pd.read_csv(yesterday_folder+'\\ref_info.csv')

#paper next id and ref next id
paperNid = len(ypaper_info)
print(paperNid)


0


In [170]:
dt = dict()
dt['id'] = []
dt['link']= []
dt['title']= []
dt['publicationName']= []
dt['affiliation']= []
dt['abstract'] = []
dt['publicationDate'] = []
dt['ref_count'] = []
dt['publicationYear'] = []


ref = dict()
ref['id'] = []
ref['ref_authors'] = []
ref['ref_publicationyear'] = []
ref['ref_title'] = []





def to_csv(result,dt,ref):
    oldlen = len(dt['id'])
    for id,paper in enumerate(result['search-results']['entry']):
        Nid = paperNid + id + oldlen
        dt['id'].append(Nid)
        dt['link'].append(paper['link'][2]['@href'])
        dt['title'].append(paper['dc:title'])
        dt['publicationName'].append(paper['prism:publicationName'])
        if 'affiliation' in paper:
            dt['affiliation'].append(paper['affiliation'])
        else:
            dt['affiliation'].append('null')
        # print(paper['link'][2]['@href'])
        page = rq.get(paper['link'][0]['@href'],headers=header)
        result_ab = page.json()
        #get abstract
        ab = result_ab['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
        #get publication date
        pubdate = result_ab['abstracts-retrieval-response']['item']['bibrecord']['head']['source']['publicationdate']
        dt['abstract'].append(ab)
        dt['publicationDate'].append(pubdate)
        dt['publicationYear'].append(pubdate['year'])
        # print(id,result_ab)
        # get reference count
        if result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail'] != None :
            refcount = result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['@refcount']
            dt['ref_count'].append(refcount)
        else:
            dt['ref_count'].append('null')
    
        #reference paper csv
        if result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail'] != None:
            allref = result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']
            for j in allref:
                # print(Nid , j)
                if type(j) is dict:
                    ref['id'].append(Nid)
                    if 'ref-fulltext' in j:
                        ref['ref_authors'].append(j['ref-fulltext'])
                    else:
                        ref['ref_authors'].append('null')
                    if "ref-publicationyear" in j['ref-info']:
                        ref['ref_publicationyear'].append(j['ref-info']['ref-publicationyear']['@first'])
                    else:
                        ref['ref_publicationyear'].append('null')
                    if 'ref-title' in j['ref-info']:
                        ref['ref_title'].append(j['ref-info']['ref-title']['ref-titletext'])
                    else:
                        ref['ref_title'].append('null')
        # break

        # df.to_csv(my_path+"\\research_csv\\paper_info\\"+"scopus_paper_"+str(id)+'.csv')

    return dt,ref


In [178]:
#publicatino year 2018 and doctype article
days_fetch = len(os.listdir('.\\research_csv'))-1
for i in range(len(pubyear)):
    search_query = '%28PUBYEAR%20%3D%20'+pubyear[i]+'%29%20AND%20%28DOCTYPE%28ar%29%29'
    itemperpage = '200'
    response = rq.get(api_url + search_query +"&count="+itemperpage  + "&start=" +str(int(itemperpage)*days_fetch) , headers = header)
    result = (response.json())
    # print(result)
    dt, ref = to_csv(result,dt,ref)
    
df = pd.DataFrame.from_dict(dt)
ref_df = pd.DataFrame.from_dict(ref)

df = pd.concat([ypaper_info , df])
ref_df = pd.concat([yref_info , ref_df])

df.to_csv(day_folder+'\\paper_info.csv')
ref_df.to_csv(day_folder+'\\ref_info.csv')

{'search-results': {'opensearch:totalResults': '2150036', 'opensearch:startIndex': '200', 'opensearch:itemsPerPage': '10', 'opensearch:Query': {'@role': 'request', '@searchTerms': '(PUBYEAR = 2018) AND (DOCTYPE(ar))', '@startPage': '200'}, 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/search/scopus?start=200&count=10&query=%28PUBYEAR+%3D+2018%29+AND+%28DOCTYPE%28ar%29%29', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'first', '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=10&query=%28PUBYEAR+%3D+2018%29+AND+%28DOCTYPE%28ar%29%29', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'prev', '@href': 'https://api.elsevier.com/content/search/scopus?start=190&count=10&query=%28PUBYEAR+%3D+2018%29+AND+%28DOCTYPE%28ar%29%29', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'next', '@href': 'https://api.elsevier.com/content/search/scopus?start=210&count=10&query=%28PUBYEAR+%3D+2018%29+AND+%28DOCTYPE%28ar%29%29'

In [172]:
# print(result)
df


,id,link,title,publicationName,affiliation,abstract,publicationDate,ref_count,publicationYear
0,0,https://www.scopus.com/inward/record.uri?partn...,Social Progress for Resilient Regions,Region,"[{'@_fa': 'true', 'affilname': 'European Inves...","© 2018, European Regional Science Association....","{'month': '12', 'year': '2018', 'date-text': {...",2,2018
1,1,https://www.scopus.com/inward/record.uri?partn...,Fashion language and translatology,Babel,"[{'@_fa': 'true', 'affilname': 'University of ...",© John Benjamins Publishing Company.My study i...,"{'month': '12', 'year': '2018', 'date-text': {...",18,2018
2,2,https://www.scopus.com/inward/record.uri?partn...,A pragmatic framework to note-taking in consec...,Babel,"[{'@_fa': 'true', 'affilname': 'An-Najah Natio...",© John Benjamins Publishing Company.This study...,"{'month': '12', 'year': '2018', 'date-text': {...",17,2018
3,3,https://www.scopus.com/inward/record.uri?partn...,An Analytic Approximation to the Density of Tw...,Recoletos Multidisciplinary Research Journal,"[{'@_fa': 'true', 'affilname': 'Agusan del Sur...","© 2018, University of San Jose-Recoletos. All ...","{'month': '12', 'year': '2018', 'date-text': {...",11,2018
4,4,https://www.scopus.com/inward/record.uri?partn...,DANCE MOTIFS ON PREHISTORIC POTTERY FROM EASTE...,Vjesnik Arheoloskog Muzeja u Zagrebu,"[{'@_fa': 'true', 'affilname': 'Centre for Exp...","© 2018, Archaeological museum in Zagreb. All r...","{'month': '12', 'year': '2018', 'date-text': {...",37,2018
...,...,...,...,...,...,...,...,...,...
1395,1395,https://www.scopus.com/inward/record.uri?partn...,Differences in cardiovascular risk and health-...,Scientific Reports,"[{'@_fa': 'true', 'affilname': 'Hospital Regio...",© The Author(s) 2024.Chronic obstructive pulmo...,"{'month': '12', 'year': '2024', 'date-text': '...",56,2024
1396,1396,https://www.scopus.com/inward/record.uri?partn...,Low-grade undifferentiated sarcoma with MEIS1:...,Diagnostic Pathology,"[{'@_fa': 'true', 'affilname': 'Northwestern U...",© The Author(s) 2024.Background: MEIS1::NCOA2 ...,"{'month': '12', 'year': '2024', 'date-text': '...",17,2024
1397,1397,https://www.scopus.com/inward/record.uri?partn...,The effect of exercise on sleep habits of chil...,BMC Pediatrics,"[{'@_fa': 'true', 'affilname': 'Lorestan Unive...",© The Author(s) 2024.Background: Adequate slee...,"{'month': '12', 'year': '2024', 'date-text': '...",50,2024
1398,1398,https://www.scopus.com/inward/record.uri?partn...,Particules Flottantes : Mutable Identity and P...,Journal of Modern Literature,"[{'@_fa': 'true', 'affilname': 'University of ...",© 2024 Indiana University Press. All rights re...,"{'month': '12', 'year': '2024', 'date-text': '...",36,2024


In [173]:
# save_file = open("savedata2.json", "w")  
# json.dump(result_ab, save_file, indent = 6)  
# save_file.close()  

495


In [174]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# import os
# # Set the download directory for the browser
# # C:\\Users\\acer\\Desktop\\code\\DataSci&DataEng\\project
# download_directory = my_path+"\\publication_recommend\\webscraping\\research_csv"
# chrome_options = webdriver.ChromeOptions()
# prefs = {"download.default_directory": download_directory}
# chrome_options.add_experimental_option("prefs", prefs)
# browser = webdriver.Chrome(options = chrome_options)
# wait = WebDriverWait(browser, 10)
# url = str(link)
# browser.get(url=url)
# time.sleep(2)
# for id,i in enumerate(result['search-results']['entry']):
#         filename = 'scopus'+str(id)
#         os.mkdir(download_directory+'\\'+filename)
#         link = i['link'][2]['@href']
#         url = str(link)
#         browser.get(url=url)
#         time.sleep(2)
#         #download research info
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module__Y4rmW']//div[@class='export-dropdown']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk']//button[@data-testid='export-to-csv']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk'][2]//label[@class='Checkbox-module__jE3jb Checkbox-module__zALPa']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk'][3]//label[@class='Checkbox-module__jE3jb Checkbox-module__zALPa']").click()
#         browser.find_element(By.XPATH,"//button[@data-testid='submit-export-button']").click()
#         #dowload research reference info
#         time.sleep(3)
#         browser.find_element(By.XPATH,"//div[@id='resultsListToolbar']//label[@for='referencePageCheckbox']").click()
#         browser.find_element(By.XPATH,"//button[@id='export_references']").click()
#         time.sleep(1)
#         browser.find_element(By.XPATH,"//ul[@id='exportList']//li[@class='radio-inline valignTop'][5]").click()
#         browser.find_element(By.XPATH,"//button[(@class = 'btn btn-primary btn-sm btnEnable') and (@id = 'exportTrigger')]").click()

In [175]:
ref_df

,id,ref_authors,ref_publicationyear,ref_title
0,0,European Commission (2015) Energy Union Packag...,2015,null
1,0,European Commission (2017) White Paper on the ...,2017,null
2,1,"Barthes, Roland. 2005. The Language of Fashion...",2005,null
3,1,"Bojović, Brankica. 2010. Leksika u kriminološk...",2010,null
4,1,"Bojović, Brankica. 2013. “Pictorial Language i...",2013,null
...,...,...,...,...
49348,1399,"Sprengnether, Madelon. “Feminist Criticism and...",2007,Feminist Criticism and Psychoanalysis
49349,1399,"Tison, Hélène. “Loss, Revision, Translation: R...",2015,"Loss, Revision, Translation: Re-Membering the ..."
49350,1399,"Waugh, Patricia. “Postmodern Theory: The Curre...",1992,Postmodern Theory: The Current Debate
49351,1399,"Weil, Elizabeth. “Pump It Up.” New York Times ...",2021,Pump It Up


In [176]:
a = {'a':[1] , 'b':[2] , 'c':[3]}
adf = pd.DataFrame.from_dict(a)

b = {'a':[12] , 'b':[33] , 'c':[31]}
bdf = pd.DataFrame.from_dict(b)

cdf = pd.concat([adf,bdf])

cdf


,a,b,c
0,1,2,3
0,12,33,31


In [177]:
df

,id,link,title,publicationName,affiliation,abstract,publicationDate,ref_count,publicationYear
0,0,https://www.scopus.com/inward/record.uri?partn...,Social Progress for Resilient Regions,Region,"[{'@_fa': 'true', 'affilname': 'European Inves...","© 2018, European Regional Science Association....","{'month': '12', 'year': '2018', 'date-text': {...",2,2018
1,1,https://www.scopus.com/inward/record.uri?partn...,Fashion language and translatology,Babel,"[{'@_fa': 'true', 'affilname': 'University of ...",© John Benjamins Publishing Company.My study i...,"{'month': '12', 'year': '2018', 'date-text': {...",18,2018
2,2,https://www.scopus.com/inward/record.uri?partn...,A pragmatic framework to note-taking in consec...,Babel,"[{'@_fa': 'true', 'affilname': 'An-Najah Natio...",© John Benjamins Publishing Company.This study...,"{'month': '12', 'year': '2018', 'date-text': {...",17,2018
3,3,https://www.scopus.com/inward/record.uri?partn...,An Analytic Approximation to the Density of Tw...,Recoletos Multidisciplinary Research Journal,"[{'@_fa': 'true', 'affilname': 'Agusan del Sur...","© 2018, University of San Jose-Recoletos. All ...","{'month': '12', 'year': '2018', 'date-text': {...",11,2018
4,4,https://www.scopus.com/inward/record.uri?partn...,DANCE MOTIFS ON PREHISTORIC POTTERY FROM EASTE...,Vjesnik Arheoloskog Muzeja u Zagrebu,"[{'@_fa': 'true', 'affilname': 'Centre for Exp...","© 2018, Archaeological museum in Zagreb. All r...","{'month': '12', 'year': '2018', 'date-text': {...",37,2018
...,...,...,...,...,...,...,...,...,...
1395,1395,https://www.scopus.com/inward/record.uri?partn...,Differences in cardiovascular risk and health-...,Scientific Reports,"[{'@_fa': 'true', 'affilname': 'Hospital Regio...",© The Author(s) 2024.Chronic obstructive pulmo...,"{'month': '12', 'year': '2024', 'date-text': '...",56,2024
1396,1396,https://www.scopus.com/inward/record.uri?partn...,Low-grade undifferentiated sarcoma with MEIS1:...,Diagnostic Pathology,"[{'@_fa': 'true', 'affilname': 'Northwestern U...",© The Author(s) 2024.Background: MEIS1::NCOA2 ...,"{'month': '12', 'year': '2024', 'date-text': '...",17,2024
1397,1397,https://www.scopus.com/inward/record.uri?partn...,The effect of exercise on sleep habits of chil...,BMC Pediatrics,"[{'@_fa': 'true', 'affilname': 'Lorestan Unive...",© The Author(s) 2024.Background: Adequate slee...,"{'month': '12', 'year': '2024', 'date-text': '...",50,2024
1398,1398,https://www.scopus.com/inward/record.uri?partn...,Particules Flottantes : Mutable Identity and P...,Journal of Modern Literature,"[{'@_fa': 'true', 'affilname': 'University of ...",© 2024 Indiana University Press. All rights re...,"{'month': '12', 'year': '2024', 'date-text': '...",36,2024
